<a href="https://colab.research.google.com/github/daddyboy77/A.I-testing/blob/master/Cartoonize%20C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Setup
!pip install imageio==2.4.1
!pip uninstall PIL
!pip uninstall Pillow
!pip install Pillow
import os
repo = "CartoonGan-tensorflow"
!git clone https://github.com/mnicnc404/CartoonGan-tensorflow.git
os.chdir(os.path.join(repo))

from IPython.display import clear_output, display, Image
!pip3 install tensorflow-gpu==2.2.0-alpha0
!git clone https://www.github.com/keras-team/keras-contrib.git \
    && cd keras-contrib \
    && python convert_to_tf_keras.py \
    && USE_TF_KERAS=1 python setup.py install

!pip3 install matlab
!pip install moviepy==1.0.2
clear_output()
import tensorflow as tf
print(tf.__version__)

2.8.2


In [2]:
#@title Link drive
#@markdown Connect google drive, folders called Real amd Cartoonized and sub-folders would be created upload vidoes you want to animate to the Real folder then proceed. The animated photos show up in the cartoonized folder.
from google.colab import drive
drive.mount('/content/drive')
clear_output()
if not os.path.exists("/content/drive/MyDrive/Real/"):
 os.mkdir("/content/drive/MyDrive/Real/")
if not os.path.exists("/content/drive/MyDrive/Cartoonized/"):
 os.mkdir("/content/drive/MyDrive/Cartoonized/")
if not os.path.exists("/content/drive/MyDrive/Cartoonized/comparison/"):
 os.mkdir("/content/drive/MyDrive/Cartoonized/comparison/")

In [3]:
#@title Use to see current files in 'Real Folder
print('Current files in the "Real" folder: ')
!ls "/content/drive/MyDrive/Real/"

Current files in the "Real" folder: 
 Alt.mp4	     Fashion.mp4     Pastor_Paul.mkv
 Comedian.mp4	     fooet.mp4	     tok.mp4
'download (4).mp4'   MirrorVid.mp4   Whatelikeis.mp4


In [19]:
#@title Process video
videoName = "fooet" #@param {type: "string"}
Video_filetype = ".mp4" #@param [".mp4", ".mkv"]
vdirectory = "/content/drive/MyDrive/Real/" + videoName + Video_filetype
vidname = vdirectory.split("/")[-1]
name = '.'.join(vidname.split(".")[:-1])
print("name: " + name)
print("vidname: " + vidname)
print("Test:" + "/content/" + vidname + "/")
 
#@title Analyze
import cv2
def count_frames(path, override=False):
    # grab a pointer to the video file and initialize the total
    # number of frames read
    video = cv2.VideoCapture(path)
    total = 0
    # if the override flag is passed in, revert to the manual
    # method of counting frames
    if override:
        total = count_frames_manual(video)
    # otherwise, let's try the fast way first
    else:
        # lets try to determine the number of frames in a video
        # via video properties; this method can be very buggy
        # and might throw an error based on your OpenCV version
        # or may fail entirely based on your which video codecs
        # you have installed
        try:
            # check if we are using OpenCV 3
            # otherwise, we are using OpenCV 2.4
            total = int(video.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))
        # uh-oh, we got an error -- revert to counting manually
        except:
            total = count_frames_manual(video)
    # release the video file pointer
    video.release()
    # return the total number of frames in the video
    return total
def count_frames_manual(video):
    # initialize the total number of frames read
    total = 0
    # loop over the frames of the video
    while True:
        # grab the current frame
        (grabbed, frame) = video.read()
     
        # check to see if we have reached the end of the
        # video
        if not grabbed:
            break
        # increment the total number of frames read
        total += 1
    # return the total number of frames in the video file
    return total
if __name__ == '__main__' :
    video = cv2.VideoCapture(vdirectory);
    # Find OpenCV version
    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
    if int(major_ver)  < 3 :
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
        print ("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
    else :
        fps = video.get(cv2.CAP_PROP_FPS)
        print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
    video.release()
print(fps)
print(count_frames(vdirectory))


from concurrent.futures import ProcessPoolExecutor, as_completed
import cv2
import multiprocessing
import os
import sys
 
 
def print_progress(iteration, total, prefix='', suffix='', decimals=3, bar_length=100):
    """
    Call in a loop to create standard out progress bar
    :param iteration: current iteration
    :param total: total iterations
    :param prefix: prefix string
    :param suffix: suffix string
    :param decimals: positive number of decimals in percent complete
    :param bar_length: character length of bar
    :return: None
    """
 
    format_str = "{0:." + str(decimals) + "f}"  # format the % done number string
    percents = format_str.format(100 * (iteration / float(total)))  # calculate the % done
    filled_length = int(round(bar_length * iteration / float(total)))  # calculate the filled bar length
    bar = '#' * filled_length + '-' * (bar_length - filled_length)  # generate the bar string
    sys.stdout.write('\r%s |%s| %s%s %s' % (prefix, bar, percents, '%', suffix)),  # write out the bar
    sys.stdout.flush()  # flush to stdout
 
 
def extract_frames(video_path, frames_dir, overwrite=False, start=-1, end=-1, every=1):
    """
    Extract frames from a video using OpenCVs VideoCapture
    :param video_path: path of the video
    :param frames_dir: the directory to save the frames
    :param overwrite: to overwrite frames that already exist?
    :param start: start frame
    :param end: end frame
    :param every: frame spacing
    :return: count of images saved
    """
 
    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible
 
    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path
 
    assert os.path.exists(video_path)  # assert the video file exists
 
    capture = cv2.VideoCapture(video_path)  # open the video using OpenCV
 
    if start < 0:  # if start isn't specified lets assume 0
        start = 0
    if end < 0:  # if end isn't specified assume the end of the video
        end = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
 
    capture.set(1, start)  # set the starting frame of the capture
    frame = start  # keep track of which frame we are up to, starting from start
    while_safety = 0  # a safety counter to ensure we don't enter an infinite while loop (hopefully we won't need it)
    saved_count = 0  # a count of how many frames we have saved
 
    while frame < end:  # lets loop through the frames until the end
 
        _, image = capture.read()  # read an image from the capture
 
        if while_safety > 500:  # break the while if our safety maxs out at 500
            break
 
        # sometimes OpenCV reads None's during a video, in which case we want to just skip
        if image is None:  # if we get a bad return flag or the image we read is None, lets not save
            while_safety += 1  # add 1 to our while safety, since we skip before incrementing our frame variable
            continue  # skip
 
        if frame % every == 0:  # if this is a frame we want to write out based on the 'every' argument
            while_safety = 0  # reset the safety count
            save_path = os.path.join(frames_dir, video_filename, "ani.{:010d}.png".format(frame))  # create the save path
            if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways
                cv2.imwrite(save_path, image)  # save the extracted image
                saved_count += 1  # increment our counter by one
 
        frame += 1  # increment our frame count
 
    capture.release()  # after the while has finished close the capture
 
    return saved_count  # and return the count of the images we saved
 
 
def video_to_frames(video_path, frames_dir, overwrite=False, every=1, chunk_size=1000):
    """
    Extracts the frames from a video using multiprocessing
    :param video_path: path to the video
    :param frames_dir: directory to save the frames
    :param overwrite: overwrite frames if they exist?
    :param every: extract every this many frames
    :param chunk_size: how many frames to split into chunks (one chunk per cpu core process)
    :return: path to the directory where the frames were saved, or None if fails
    """
 
    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible
 
    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path
 
    # make directory to save frames, its a sub dir in the frames_dir with the video name
    os.makedirs(os.path.join(frames_dir, video_filename), exist_ok=True)
 
    capture = cv2.VideoCapture(video_path)  # load the video
    total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))  # get its total frame count
    capture.release()  # release the capture straight away
 
    if total < 1:  # if video has no frames, might be and opencv error
        print("Video has no frames. Check your OpenCV + ffmpeg installation, can't read videos!!!\n"
              "You may need to install OpenCV by source not pip")
        return None  # return None
 
    frame_chunks = [[i, i+chunk_size] for i in range(0, total, chunk_size)]  # split the frames into chunk lists
    frame_chunks[-1][-1] = min(frame_chunks[-1][-1], total-1)  # make sure last chunk has correct end frame
 
    prefix_str = "Extracting frames from {}".format(video_filename)  # a prefix string to be printed in progress bar
 
    # execute across multiple cpu cores to speed up processing, get the count automatically
    with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
 
        futures = [executor.submit(extract_frames, video_path, frames_dir, overwrite, f[0], f[1], every)
                   for f in frame_chunks]  # submit the processes: extract_frames(...)
 
        for i, f in enumerate(as_completed(futures)):  # as each process completes
            print_progress(i, len(frame_chunks)-1, prefix=prefix_str, suffix='Complete')  # print it's progress
 
    return os.path.join(frames_dir, video_filename)  # when done return the directory containing the frames
 
Chunk_Slider = 25 #@param {type:"slider", min:0, max:1000, step:25}
Comparison = False #@param {type:"boolean"}
#@markdown Just the name of the video without any extra spaces at the end, no need to add mp4
#@markdown 

#@markdown Comparison means it'll put the real video side by side with the fake or something like that
if __name__ == '__main__':
    # test it
    video_to_frames(video_path=vdirectory, frames_dir="/content/", overwrite=False, every=1, chunk_size=Chunk_Slider)
from moviepy.editor import *
if not os.path.exists("/content/sound/"):
 os.mkdir("/content/sound/")

video = VideoFileClip(vdirectory) 
audio = video.audio 
audioname = "/content/sound/" + name + ".mp3"
# if not os.path.exists(audioname):
#  os.mkdir(audioname)
audio.write_audiofile(audioname)
#lear_output()

name: fooet
vidname: fooet.mp4
Test:/content/fooet.mp4/
Frames per second using video.get(cv2.CAP_PROP_FPS) : 14.012039975089706
14.012039975089706
42
Extracting frames from fooet.mp4 |####################################################################################################| 100.000% CompleteMoviePy - Writing audio in /content/sound/fooet.mp3


MoviePy - Done.


In [ ]:
Comparison = False #@param {type:"boolean"}

In [ ]:
#@title Help
!python3 cartoonize.py --help

In [20]:
#@title Animation style
import math
styles = "AnimeGan2" #@param ["shinkai", "hayao", "hosoda", "paprika", "AnimeGan2"]
Outpudir = "/content/" + videoName
Inputdir = Outpudir + Video_filetype
sn = "/content/" + styles
rfpsl = math.floor(fps)

In [ ]:
from io import BytesIO
import torch
from PIL import Image

import ipywidgets as widgets
import IPython.display as display
from google.colab import files

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)
image_format = "png" 

if not os.path.exists(Outpudir + "/" + styles + "/"):
  os.mkdir(Outpudir + "/" + styles + "/")
# import required module

# assign directory
# directory = '/content/Alt.mp4/'

# iterate over files in
# that directory
for filename in os.listdir(Inputdir):
  f = os.path.join(Inputdir, filename)
# checking if it is a file
  if os.path.isfile(f):
    img = Image.open(f).convert("RGB")
    out = face2paint(model, img)
    #print(Outpudir + "/" + styles + "/" + filename)
    out.save(Outpudir + "/" + styles + "/" + filename)
clear_output()

In [32]:
#@title Cartoonize retain resolution (Higher quality but slow)
!rm -rf {sn}
!rm -rf "/content/comparison"
small = False
import os
from io import BytesIO
import torch
from PIL import Image
import ipywidgets as widgets
import IPython.display as display
from google.colab import files

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)
image_format = "png" 

if(styles != "AnimeGan2"):
  if(Comparison == True):
    !python cartoonize.py \
        --styles {styles} \
        --batch_size {rfpsl} \
        --comparison_view horizontal \
        --input_dir {Inputdir} \
        --keep_original_size \
        --output_dir {Outpudir}
  else:
      !python cartoonize.py \
        --styles {styles} \
        --batch_size {rfpsl} \
        --input_dir {Inputdir} \
        --keep_original_size \
        --output_dir {Outpudir}
else:
  if not os.path.exists(Outpudir + "/" + styles + "/"):
    os.mkdir(Outpudir + "/")
    os.mkdir(Outpudir + "/" + styles + "/")
  # import required module

  # assign directory
  # directory = '/content/Alt.mp4/'

  # iterate over files in
  # that directory
  for filename in os.listdir(Inputdir):
    f = os.path.join(Inputdir, filename)
    print('Processing image ', f, ' of ', len(os.listdir(Inputdir)))
  # checking if it is a file
    if os.path.isfile(f):
      img = Image.open(f).convert("RGB")
      out = face2paint(model, img)
      print(Outpudir + "/" + styles + "/" + filename)
      out.save(Outpudir + "/" + styles + "/" + filename)
  clear_output()
#clear_output()

In [ ]:
#@title Cartoonize resize resolution (Lower quality but faster)
!rm -rf {sn}
small = True
!rm -rf "/content/comparison"
if(Comparison == True):
  !python3 cartoonize.py \
      --styles {styles} \
      --batch_size {rfpsl} \
      --comparison_view horizontal \
      --input_dir {Inputdir} \
      --output_dir "/content/"
else:
    !python3 cartoonize.py \
      --styles {styles} \
      --batch_size {rfpsl} \
      --input_dir {Inputdir} \
      --output_dir "/content/"

In [33]:
#@title Download folder of animated images
#@markdown You could import the images as an image sequence to be imported into animation software such as opentoonz for modification.
!zip -r {videoName} {Outpudir}
from google.colab import files
videohezip = videoName + ".zip"
clear_output()
files.download(videohezip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title For downloading folders
Folder_To_Download = "Comedian.mp4" #@param {type: "string"}
dfold = "/content/" + Folder_To_Download
zipname = "/content/" + Folder_To_Download +".zip"
!zip -r {zipname} {dfold}
from google.colab import files
files.download(zipname)

In [13]:
small = False

In [15]:
from IPython.display import clear_output, display, Image

In [34]:
#@title Compile animated video
import cv2
import numpy as np
import os
import random 
from os.path import isfile, join
import shutil
 
numberList = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41]
entil = random.choice(numberList)

if(small == True):
  add = " rough"
else:
  add = " synced"
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        #print(img)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
def main():
    pathIn= Outpudir + "/" + styles + "/"
    pathOut = "/content/drive/MyDrive/Cartoonized/" + name + "tmp" + styles + add + ".mp4"
    convert_frames_to_video(pathIn, pathOut, fps)

if __name__=="__main__":
    main()
pathIn= Outpudir + "/" + styles + "/"
pathOut = "/content/drive/MyDrive/Cartoonized/" + name + "tmp" + styles + add + ".mp4"
npathOut = "/content/drive/MyDrive/Cartoonized/" + name + " " + styles + add + ".mp4"


patht = "/content/drive/MyDrive/Cartoonized/comparison/" + name + "tmp" + styles + add + " comparison" + ".mp4"
npatht = "/content/drive/MyDrive/Cartoonized/comparison/" + name + " " + styles + add + " comparison" + ".mp4"
if(Comparison == True):
   convert_frames_to_video("/content/comparison/", patht, fps)

clear_output()
from moviepy.editor import *


videoclip = VideoFileClip(pathOut)
audioclip = AudioFileClip(audioname)
new_audioclip = CompositeAudioClip([audioclip])
videoclip.audio = new_audioclip
videoclip.write_videofile(npathOut)


delete_filepath = pathOut
open(delete_filepath, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
os.remove(delete_filepath)

if(Comparison == True):
  videoclipe = VideoFileClip(patht)
  audioclipe = AudioFileClip(audioname)
  new_audioclipe = CompositeAudioClip([audioclipe])
  videoclipe.audio = new_audioclipe
  videoclipe.write_videofile(npatht)
  delete_filepathd = patht
  open(delete_filepathd, 'w').close() 
  os.remove(delete_filepathd)



Moviepy - Building video /content/drive/MyDrive/Cartoonized/fooet AnimeGan2 synced.mp4.
MoviePy - Writing audio in fooet AnimeGan2 syncedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Cartoonized/fooet AnimeGan2 synced.mp4



t:  69%|██████▉   | 29/42 [00:00<00:00, 289.24it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.7/dist-packages/moviepy/video/io/ffmpeg_reader.py:130: UserWarning: Warning: in file /content/drive/MyDrive/Cartoonized/fooettmpAnimeGan2 synced.mp4, 786432 bytes wanted but 0 bytes read,at frame 41/42, at time 2.93/2.93 sec. Using the last valid frame instead.
  UserWarning)



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Cartoonized/fooet AnimeGan2 synced.mp4


In [35]:
#@title Download animated video
print(npatht)
if os.path.exists(npathOut):
  files.download(npathOut)
if os.path.exists(npatht):
  files.download(npatht)

/content/drive/MyDrive/Cartoonized/comparison/fooet AnimeGan2 synced comparison.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Clear before new video
!rm -rf "/content/paprika"
!rm -rf "/content/hosoda"
!rm -rf "/content/hayao"
!rm -rf "/content/shinkai"
!rm -rf "/content/comparison"
!rm -rf {Inputdir}
clear_output()